In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf

# # Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF

KeyError: 'PROJ_LIB'

In [ ]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

# Read in the wrfout file using the netCDF4.Dataset method (I think you can also do this with an xarray method)
netcdf_data = netCDF4.Dataset(datapath)

In [ ]:
# Create an xarray.Dataset from the wrf qurery_variables.
query_variables = [
            'Times',
            'T2',
            'U10',
            'V10',
            'SWDDNI',
            'SWDDIF'
            ]
first = True
for key in query_variables:
    var = wrf.getvar(netcdf_data, key, timeidx=wrf.ALL_TIMES)
    if first:
        solar_data = var
        first = False
    else:
        solar_data = xr.merge([solar_data, var])

variables = {
            'times': 'times',
            'XLAT': 'lat',
            'XLONG': 'lon',
            'T2': 'temp_air',
            'U10': 'wind_speed_u',
            'V10': 'wind_speed_v',
            'SWDDNI': 'dni',
            'SWDDIF': 'dhi' 
            }
solar_data = xr.Dataset.rename(solar_data, variables)
solar_data = xr.Dataset.reset_coords(solar_data, ['XTIME'], drop=True)
times = solar_data.times
solar_data = xr.Dataset.set_coords(solar_data, ['times'])
solar_data = xr.Dataset.reset_coords(solar_data, ['times'], drop=True)
ntimes = solar_data.sizes['Time']
nlat = solar_data.sizes['south_north']
nlon = solar_data.sizes['west_east']
solar_data

In [ ]:
# Explore how the WRF forecast model behaves
fm = WRF()
wind_speed = fm.uv_to_speed(solar_data)
temp_air = fm.kelvin_to_celsius(solar_data['temp_air'])

In [ ]:
# Convert xarray Datasets to a pandas DataFrames
solar_data = solar_data.to_dataframe()
times = times.to_dataframe()
solar_data

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from mpl_toolkits.basemap import Basemap

# Open the NetCDF file
ncfile = netcdf_data

# Get the sea level pressure
slp = wrf.getvar(ncfile, "slp")

# Smooth the sea level pressure since it tends to be noisy near the
# mountains
smooth_slp = wrf.smooth2d(slp, 3, cenweight=4)

# Get the latitude and longitude points
lats, lons = wrf.latlon_coords(slp)

# Get the basemap object
bm = wrf.get_basemap(slp)

# Create a figure
fig = plt.figure(figsize=(12,9))

# Add geographic outlines
bm.drawcoastlines(linewidth=0.25)
bm.drawstates(linewidth=0.25)
bm.drawcountries(linewidth=0.25)

# Convert the lats and lons to x and y.  Make sure you convert the lats and
# lons to numpy arrays via to_np, or basemap crashes with an undefined
# RuntimeError.
x, y = bm(wrf.to_np(lons), wrf.to_np(lats))

# Draw the contours and filled contours
bm.contour(x, y, wrf.to_np(smooth_slp), 10, colors="black")
bm.contourf(x, y, wrf.to_np(smooth_slp), 10, cmap=get_cmap("jet"))

# Add a color bar
plt.colorbar(shrink=.62)

plt.title("Sea Level Pressure (hPa)")

plt.show()